In [ ]:
# Cell 1: Initialize OpenAI Client and Load Environment
import os
from dotenv import load_dotenv
from openai import OpenAI

# Load environment variables from .env file
load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")

# Initialize OpenAI client with API key
openai_client = OpenAI(api_key=api_key)

print("✓ Environment loaded successfully")
print(f"✓ OpenAI API key loaded: {api_key[:7]}...")
print(f"✓ OpenAI client initialized")

In [ ]:
# Cell 2: Import additional libraries
import chromadb
from chromadb.utils import embedding_functions
from pathlib import Path
import json
from typing import List, Dict, Any, Optional

print("✓ All libraries imported successfully")

In [ ]:
# Cell 3: Initialize ChromaDB with OpenAI embeddings
chroma_client = chromadb.Client()

# Create OpenAI embedding function using the API key
openai_ef = embedding_functions.OpenAIEmbeddingFunction(
    api_key=api_key,
    model_name="text-embedding-3-small"
)


collection = chroma_client.get_or_create_collection(
    name="cyber_sachet_docs",
    embedding_function=openai_ef,
    metadata={"description": "Cyber awareness and Nepal cyber laws"}
)

print(f"ChromaDB initialized")
print(f"Collection 'cyber_sachet_docs' ready")
print(f"Current documents in collection: {collection.count()}")

ChromaDB initialized
Collection 'cyber_sachet_docs' ready
Current documents in collection: 0


In [4]:

def load_and_index_documents():
    """Load documents from docs folder and add them to vector database"""
    docs_path = Path("docs")
    
    documents = []
    metadatas = []
    ids = []
    
    doc_id = 0
    for doc_file in docs_path.glob("*.txt"):
        print(f"📄 Loading: {doc_file.name}")
        
        with open(doc_file, 'r', encoding='utf-8') as f:
            content = f.read()
        
      
        chunks = [chunk.strip() for chunk in content.split('\n\n') if chunk.strip()]
        
        for i, chunk in enumerate(chunks):
            if len(chunk) > 50:  
                documents.append(chunk)
                metadatas.append({
                    "source": doc_file.name,
                    "chunk_id": i,
                    "doc_type": "cyber_law" if "act" in doc_file.name.lower() else "awareness"
                })
                ids.append(f"{doc_file.stem}_chunk_{i}")
                doc_id += 1
    
   
    if documents:
        collection.add(
            documents=documents,
            metadatas=metadatas,
            ids=ids
        )
        print(f"\n Indexed {len(documents)} chunks from {len(list(docs_path.glob('*.txt')))} documents")
        print(f" Total documents in collection: {collection.count()}")
    else:
        print("No documents found to index")


load_and_index_documents()

📄 Loading: cyber_awareness_guide.txt
📄 Loading: nepal_digital_security_act_2024.txt
📄 Loading: nepal_information_technology_act_2063.txt


AuthenticationError: Error code: 401 - {'error': {'message': 'Incorrect API key provided: <openai.********************************230>. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}} in add.

## Question 2: Search Tool Implementation

### Tool Description:
The **`semantic_search_tool`** performs vector-based semantic search on cyber security documents using ChromaDB and OpenAI embeddings.

### What It Does:
- Converts user queries into embeddings using OpenAI's text-embedding-3-small model
- Searches through indexed cyber law documents and awareness guides
- Returns the most relevant document chunks based on semantic similarity
- Supports filtering by document type (laws vs awareness)
- Provides relevance scores for each result

### Function Signature:
```python
def semantic_search_tool(
    query: str,
    n_results: int = 5,
    doc_type_filter: str = None
) -> List[Dict[str, Any]]
```

**Inputs:**
- `query` (str): The search query from the user
- `n_results` (int, default=5): Number of results to return
- `doc_type_filter` (str, optional): Filter by "cyber_law" or "awareness"

**Outputs:**
- List[Dict] containing:
  - `content`: The text chunk from the document
  - `source`: Original document filename
  - `doc_type`: Type of document (cyber_law or awareness)
  - `relevance_score`: Similarity score (0-1, higher is more relevant)
  - `chunk_id`: Position in the original document

### Challenges Faced:
1. **Chunk Size Optimization**: Balancing between too small (loss of context) and too large (reduced precision)
2. **Encoding Issues**: Handling UTF-8 characters in Nepal-specific documents
3. **Metadata Management**: Tracking document source and maintaining relationships between chunks
4. **Relevance Scoring**: Converting ChromaDB distance metrics to intuitive relevance scores
5. **API Rate Limits**: Managing OpenAI API calls for embedding generation during indexing

In [ ]:

from typing import List, Dict, Any, Optional

class CyberSearchTool:
    """
    Semantic Search Tool for Cyber Sachet Agent
    
    This tool enables vector-based semantic search across cyber security
    documents and Nepal's cyber law documentation.
    """
    
    def __init__(self, collection, embedding_function):
        self.collection = collection
        self.embedding_function = embedding_function
    
    def semantic_search_tool(
        self, 
        query: str, 
        n_results: int = 5,
        doc_type_filter: Optional[str] = None
    ) -> List[Dict[str, Any]]:
        """
        Perform semantic search on cyber security documents.
        
        Args:
            query (str): The search query
            n_results (int): Number of results to return (default: 5)
            doc_type_filter (str, optional): Filter by "cyber_law" or "awareness"
        
        Returns:
            List[Dict]: List of relevant document chunks with metadata
                - content: The document text
                - source: Original filename
                - doc_type: Type of document
                - relevance_score: Similarity score (0-1)
                - chunk_id: Position in document
        """
        
        # Build query parameters
        query_params = {
            "query_texts": [query],
            "n_results": n_results
        }
        
        # Add filter if specified
        if doc_type_filter:
            query_params["where"] = {"doc_type": doc_type_filter}
        
        # Perform vector search
        results = self.collection.query(**query_params)
        
        # Format results
        search_results = []
        if results['documents'][0]:
            for i in range(len(results['documents'][0])):
                # Calculate relevance score (1 - normalized distance)
                distance = results['distances'][0][i] if 'distances' in results else 0
                relevance_score = max(0, 1 - distance)
                
                search_results.append({
                    'content': results['documents'][0][i],
                    'source': results['metadatas'][0][i]['source'],
                    'doc_type': results['metadatas'][0][i]['doc_type'],
                    'relevance_score': round(relevance_score, 3),
                    'chunk_id': results['metadatas'][0][i]['chunk_id']
                })
        
        return search_results
    
    def search_laws_only(self, query: str, n_results: int = 3) -> List[Dict[str, Any]]:
        """Search only in cyber law documents"""
        return self.semantic_search_tool(query, n_results, "cyber_law")
    
    def search_awareness_only(self, query: str, n_results: int = 3) -> List[Dict[str, Any]]:
        """Search only in awareness documents"""
        return self.semantic_search_tool(query, n_results, "awareness")
    
    def get_all_sources(self) -> List[str]:
        """Get list of all indexed document sources"""
        all_data = self.collection.get()
        sources = set([meta['source'] for meta in all_data['metadatas']])
        return sorted(list(sources))

# Initialize the search tool
search_tool = CyberSearchTool(collection, openai_ef)

print("=" * 70)
print("🔍 SEARCH TOOL IMPLEMENTATION COMPLETE")
print("=" * 70)
print("\n📚 Indexed Documents:")
for i, source in enumerate(search_tool.get_all_sources(), 1):
    print(f"   {i}. {source}")

print("\n🛠️  Available Search Functions:")
print("   • semantic_search_tool() - General semantic search")
print("   • search_laws_only() - Search cyber law documents")
print("   • search_awareness_only() - Search awareness guides")
print("   • get_all_sources() - List all indexed documents")

print("\n" + "=" * 70)
print("Testing the Search Tool...")
print("=" * 70)

# Test the search tool
test_query = "What are the penalties for cyber crimes in Nepal?"
print(f"\n🔎 Query: '{test_query}'")
print(f"📊 Searching...")

results = search_tool.semantic_search_tool(test_query, n_results=3)

print(f"\n✅ Found {len(results)} relevant results:\n")

for i, result in enumerate(results, 1):
    print(f"Result {i}:")
    print(f"  📄 Source: {result['source']}")
    print(f"  🏷️  Type: {result['doc_type']}")
    print(f"  ⭐ Relevance: {result['relevance_score']}")
    print(f"  📝 Content Preview: {result['content'][:150]}...")
    print()

## Question 3: Agent Creation with Tools

### Agent Instructions/Prompt:

The Cyber Sachet Agent is created with the following system prompt and capabilities:

**System Prompt:**
```
You are Cyber Sachet, an AI assistant specializing in cyber security awareness 
and Nepal's cyber laws (IT Act 2063, Digital Security Act 2024).

Your mission:
1. Provide accurate, helpful information about cyber security
2. Explain Nepal's cyber laws in clear, accessible language
3. Offer practical security advice and best practices
4. Use the semantic_search_tool to find relevant information
5. Always cite sources when referencing specific laws
6. Be educational, empathetic, and solution-oriented

When answering:
- First use the search tool to gather relevant context
- Synthesize information from multiple sources
- Provide actionable advice
- Cite the document source for legal information
- If uncertain, clearly state limitations
```

**Available Tools:**
1. `semantic_search_tool()` - Semantic search across all documents
2. `search_laws_only()` - Search only cyber law documents
3. `search_awareness_only()` - Search only awareness guides

**Agent Architecture:**
- Uses RAG (Retrieval Augmented Generation) approach
- OpenAI GPT-4o-mini for responses
- ChromaDB for vector storage and retrieval
- Context window: Up to 5 relevant document chunks per query

In [ ]:
# Cell 6: QUESTION 3 - Create the Agent with Search Tools

class CyberSachetAgent:
    """
    Cyber Sachet Agent - AI Assistant for Cyber Security and Nepal Cyber Laws
    
    Uses RAG (Retrieval Augmented Generation) with semantic search tools
    """
    
    def __init__(self, search_tool: CyberSearchTool, openai_client):
        self.search_tool = search_tool
        self.client = openai_client
        
        # Agent's system prompt
        self.system_prompt = """You are Cyber Sachet, an AI assistant specializing in cyber security awareness 
and Nepal's cyber laws (IT Act 2063, Digital Security Act 2024).

Your mission:
1. Provide accurate, helpful information about cyber security
2. Explain Nepal's cyber laws in clear, accessible language
3. Offer practical security advice and best practices
4. Always cite sources when referencing specific laws or regulations
5. Be educational, empathetic, and solution-oriented

When answering questions:
- Synthesize information from the provided context documents
- Provide actionable, practical advice
- Cite the specific document source for legal information (e.g., "According to Nepal IT Act 2063...")
- Use clear, simple language that non-technical users can understand
- If the context doesn't contain the answer, use your general knowledge but clearly state that
- For cyber security tips, prioritize practical, implementable advice

Remember: Your goal is to educate and empower users about cyber security and their legal rights/responsibilities."""
    
    def query(self, user_question: str, n_context_docs: int = 5, verbose: bool = True) -> Dict[str, Any]:
        """
        Query the agent with a question.
        
        Args:
            user_question (str): User's question
            n_context_docs (int): Number of context documents to retrieve
            verbose (bool): Print search process details
        
        Returns:
            Dict containing the answer, sources used, and metadata
        """
        
        if verbose:
            print(f"🔍 Searching knowledge base for: '{user_question}'")
        
        # Use the search tool to retrieve relevant context
        search_results = self.search_tool.semantic_search_tool(
            query=user_question,
            n_results=n_context_docs
        )
        
        if verbose:
            print(f"📚 Retrieved {len(search_results)} relevant documents")
            for i, r in enumerate(search_results[:3], 1):
                print(f"   {i}. {r['source']} (relevance: {r['relevance_score']})")
        
        # Build context from search results
        context_parts = []
        sources_used = set()
        
        for result in search_results:
            context_parts.append(
                f"[Source: {result['source']}, Type: {result['doc_type']}, Relevance: {result['relevance_score']}]\n"
                f"{result['content']}"
            )
            sources_used.add(result['source'])
        
        context = "\n\n---\n\n".join(context_parts)
        
        # Build the user message with context
        user_message = f"""Based on the following context from cyber security documents and Nepal's cyber laws, 
please answer the user's question comprehensively.

CONTEXT DOCUMENTS:
{context}

USER QUESTION: {user_question}

Please provide a comprehensive, well-structured answer. If referencing specific laws or regulations, 
cite the source document. Make your answer practical and actionable."""
        
        # Call OpenAI API
        messages = [
            {"role": "system", "content": self.system_prompt},
            {"role": "user", "content": user_message}
        ]
        
        if verbose:
            print("🤖 Generating response...\n")
        
        response = self.client.chat.completions.create(
            model="gpt-4o-mini",
            messages=messages,
            temperature=0.7,
            max_tokens=1000
        )
        
        answer = response.choices[0].message.content
        
        return {
            "question": user_question,
            "answer": answer,
            "sources_used": list(sources_used),
            "num_context_docs": len(search_results),
            "search_results": search_results
        }
    
    def quick_query(self, user_question: str) -> str:
        """Quick query that just returns the answer"""
        result = self.query(user_question, verbose=False)
        return result["answer"]

# Initialize the agent
agent = CyberSachetAgent(search_tool, openai_client)

print("=" * 70)
print("🤖 CYBER SACHET AGENT INITIALIZED")
print("=" * 70)
print("\n✅ Agent is ready with:")
print("   • Semantic search capabilities")
print("   • Access to Nepal cyber law documents")
print("   • Cyber security awareness knowledge")
print("   • RAG-based response generation")
print("\n" + "=" * 70)

## Example Interactions

Below are example interactions showing how the agent uses the search tool to answer questions.

In [ ]:
# Cell 7: Example Interaction 1 - Legal Question

print("=" * 70)
print("EXAMPLE INTERACTION 1: Legal Question")
print("=" * 70)
print()

question1 = "What are the penalties for hacking and unauthorized access in Nepal?"

print(f"👤 User: {question1}\n")

result1 = agent.query(question1, n_context_docs=5)

print(f"💬 Cyber Sachet Agent:\n{result1['answer']}\n")

print("━" * 70)
print(f"📊 Metadata:")
print(f"   Sources Used: {', '.join(result1['sources_used'])}")
print(f"   Context Documents Retrieved: {result1['num_context_docs']}")
print(f"   Top Relevance Score: {result1['search_results'][0]['relevance_score']}")
print("=" * 70)

In [ ]:
# Cell 8: Example Interaction 2 - Security Awareness Question

print("\n" + "=" * 70)
print("EXAMPLE INTERACTION 2: Security Awareness Question")
print("=" * 70)
print()

question2 = "How can I protect myself from phishing attacks?"

print(f"👤 User: {question2}\n")

result2 = agent.query(question2, n_context_docs=4)

print(f"💬 Cyber Sachet Agent:\n{result2['answer']}\n")

print("━" * 70)
print(f"📊 Metadata:")
print(f"   Sources Used: {', '.join(result2['sources_used'])}")
print(f"   Context Documents Retrieved: {result2['num_context_docs']}")
print("=" * 70)

In [ ]:
# Cell 9: Example Interaction 3 - Mixed Question (Law + Awareness)

print("\n" + "=" * 70)
print("EXAMPLE INTERACTION 3: Mixed Question (Law + Awareness)")
print("=" * 70)
print()

question3 = "What should I do if I become a victim of cybercrime in Nepal?"

print(f"👤 User: {question3}\n")

result3 = agent.query(question3, n_context_docs=5)

print(f"💬 Cyber Sachet Agent:\n{result3['answer']}\n")

print("━" * 70)
print(f"📊 Metadata:")
print(f"   Sources Used: {', '.join(result3['sources_used'])}")
print(f"   Context Documents Retrieved: {result3['num_context_docs']}")
print(f"   Document Types: {set([r['doc_type'] for r in result3['search_results']])}")
print("=" * 70)

In [ ]:
# Cell 10: Demonstrate Tool Usage - Search-Only Examples

print("\n" + "=" * 70)
print("DEMONSTRATING SEARCH TOOL USAGE")
print("=" * 70)

# Example 1: General search
print("\n1️⃣ General Semantic Search:")
print("Query: 'data privacy'")
privacy_results = search_tool.semantic_search_tool("data privacy", n_results=3)
for i, r in enumerate(privacy_results, 1):
    print(f"\n   Result {i}:")
    print(f"   Source: {r['source']}")
    print(f"   Relevance: {r['relevance_score']}")
    print(f"   Preview: {r['content'][:100]}...")

# Example 2: Laws only
print("\n\n2️⃣ Search Laws Only:")
print("Query: 'punishment'")
law_results = search_tool.search_laws_only("punishment", n_results=2)
for i, r in enumerate(law_results, 1):
    print(f"\n   Result {i}:")
    print(f"   Source: {r['source']}")
    print(f"   Relevance: {r['relevance_score']}")
    print(f"   Preview: {r['content'][:100]}...")

# Example 3: Awareness only
print("\n\n3️⃣ Search Awareness Guides Only:")
print("Query: 'password security'")
awareness_results = search_tool.search_awareness_only("password security", n_results=2)
for i, r in enumerate(awareness_results, 1):
    print(f"\n   Result {i}:")
    print(f"   Source: {r['source']}")
    print(f"   Relevance: {r['relevance_score']}")
    print(f"   Preview: {r['content'][:100]}...")

print("\n" + "=" * 70)
print("✅ ALL DEMONSTRATIONS COMPLETE")
print("=" * 70)

## Summary

### Question 2: Search Tool Implementation ✅

**Tool Name:** `semantic_search_tool`

**Function Signature:**
```python
def semantic_search_tool(
    query: str, 
    n_results: int = 5,
    doc_type_filter: Optional[str] = None
) -> List[Dict[str, Any]]
```

**Inputs:**
- `query`: User's search query
- `n_results`: Number of results (default: 5)
- `doc_type_filter`: Optional filter for document type

**Outputs:**
- List of dictionaries with: content, source, doc_type, relevance_score, chunk_id

**Challenges:**
1. Chunk size optimization
2. UTF-8 encoding for Nepali documents
3. Metadata management across chunks
4. Distance to relevance score conversion
5. API rate limiting during indexing

---

### Question 3: Agent with Tools ✅

**Agent Name:** Cyber Sachet Agent

**System Prompt:** Educational AI assistant for cyber security and Nepal cyber laws with RAG capabilities

**Tools Integrated:**
- `semantic_search_tool()` - General search
- `search_laws_only()` - Law-specific search
- `search_awareness_only()` - Awareness-specific search

**Example Interactions:** Demonstrated with 3 different query types showing tool usage, sources, and relevance scores

**Architecture:** RAG approach using ChromaDB + OpenAI embeddings + GPT-4o-mini